In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import torch

model_id="google/gemma-3-1b-it"
# model_id="Qwen/Qwen3Guard-Gen-4B"
device="cpu"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, dtype="auto", device_map=device)

# Try restarting your kernel if having issues


In [ ]:
generation_pipeline = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

generation_pipeline("What is your name?", max_new_tokens=40)

In [ ]:
input_prompt = [
    "What is 1 + 1?",
    "The capital of the US is",
    "gibberish"
]

tokenized = tokenizer(input_prompt, return_tensors="pt", padding=True).to(device)

print(tokenized["input_ids"])

In [ ]:
prompt_template = [
    {
        "role": "system",
        "content": "You are a helpful AI assitant that helps engineers and real estate developers"
    },
    {
        "role": "user",
        "content": "Where does the sun rise?"
    },
    {
        "role": "assistant",
        "content": "A developer is"
    }
]

tokenizer.pad_token = tokenizer.eos_token

tokenized = tokenizer.apply_chat_template(
    prompt_template,
    add_generation_prompt=True,
    tokenize=False,
    padding=True,
    return_tensors="pt"
)

print(tokenized)
# Note, google/gemma-3-1b-it does not use system roles. The system prompt is injected directly into the first user message

In [170]:
prompt_template = [
    {
        "role": "system",
        "content": "You are a helpful AI assitant that helps engineers and real estate developers"
    },
    {
        "role": "user",
        "content": "Calculate the design occupant load for a 500 sq ft Library Reading Room in a new university building in Manhattan, per NYC 2022 Building Code."
    },
    {
        "role": "assistant",
        "content": "Reasoning: "
    }
]

tokenizer.pad_token = tokenizer.eos_token

tokenized = tokenizer.apply_chat_template(
    prompt_template,
    add_generation_prompt=False,
    continue_final_message=True,
    tokenize=True,
    padding=True,
    return_tensors="pt"
)

print(tokenized)
# Note, google/gemma-3-1b-it does not use system roles. The system prompt is injected directly into the first user message

tensor([[     2,    105,   2364,    107,   3048,    659,    496,  11045,  12498,
           1188,  42563,    600,   7351,  22072,    532,   1759,   9350,  18270,
            108,  50702,    506,   1702, 109616,   3711,    573,    496, 236743,
         236810, 236771, 236771,  18288,  12110,  13826,  22033,  18120,    528,
            496,    861,  10467,   3788,    528,  35549, 236764,    810,  46585,
         236743, 236778, 236771, 236778, 236778,  16087,   8739, 236761,    106,
            107,    105,   4368,    107,  47875,    522, 236787]])


In [ ]:
out = model.generate(tokenized.to(device), max_new_tokens=80)
decoded = tokenizer.batch_decode(out)
print(decoded[0])

In [ ]:
text = "Hello how are"
input_ids = tokenizer([text], return_tensors="pt")["input_ids"].to(device)
out = model(input_ids)

In [ ]:
import torch.nn as nn
probability_dist = nn.Softmax()(out.logits[0, -1])


In [ ]:
probability_dist[611]

In [ ]:
max(probability_dist)
probability_dist.argmax()

In [ ]:
tokenizer.convert_ids_to_tokens(223130)

In [ ]:
tokenizer.convert_tokens_to_ids("▁you")
# tokenizer.vocab["you"] 

# probability_dist.shape
# torch.max(probability_dist)

# for key, val in probability_dist.items():
#     if val > 0.1:
#         print(key)
#         print(val)


In [ ]:
VOCABULARY_SIZE_DIMENSION = -1
BATCH_INDEX = 0

out.logits.argmax(axis=VOCABULARY_SIZE_DIMENSION)[BATCH_INDEX, -1]
# out.logits.argmax(axis=-1)[0, -1]
# out.logits.shape

In [ ]:
input_ids = tokenized[:, :-1]
target_ids = tokenized[:, 1:]

print(input_ids)
print(target_ids)

In [ ]:
answer = prompt_template[-1]["content"]
print(answer)
labels_tokenized = tokenizer([" " + answer], add_special_tokens=False, return_tensors="pt")["input_ids"]
print(labels_tokenized)


In [166]:
labels_tokenized = tokenizer([" " + answer + tokenizer.eos_token],
                             add_special_tokens=False, return_tensors="pt", padding="max_length", max_length=target_ids.shape[1])["input_ids"]
labels_tokenized

tensor([[     1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1, 121817, 236787, 236743,      1]])

In [168]:
labels_tokenized_fixed = torch.where(labels_tokenized == tokenizer.pad_token_id, -100, labels_tokenized)
labels_tokenized_fixed[:,-1] = tokenizer.eos_token_id
labels_tokenized_fixed

tensor([[  -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100, 121817, 236787, 236743,      1]])

In [ ]:
response = """Reasoning: Per NYC Building Code Table 1004.5, the occupant load factor for library reading rooms is 1 person per 50 net sq ft. 
Calculation: 500 / 50 = 10.0. 

Answer: 10 occupants"""
response

In [ ]:
def generate_input_output_pair(prompt, target_responses):
    chat_templates = tokenizer.apply_chat_template(
        prompt,
        continue_final_message=True,
        tokenize=False,
    )
    print(chat_templates)
    # full_response_text = [
    #     (chat_template + " " + target_response + tokenizer.eos)
    # ]


generate_input_output_pair(prompt_template, )